In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#import geopandas as gpd # geodata processing
# Get geolocation using geocoder
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='colombia_covid_19_pipe', timeout=None)
# Https requests
import requests
import unidecode
# Dates
import datetime
from datetime import date
from calendar import day_name, month_name

import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])
# PDFMiner pdfminer.six
try:
    from pdfminer.high_level import extract_text
except Exception:
    install('pdfminer.six')
    from pdfminer.high_level import extract_text

import tempfile
import os

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Colombia Covid 19 Pipeline
Dataset is obtained from [Instituto Nacional de Salud](https://www.ins.gov.co/Noticias/Paginas/Coronavirus.aspx) daily report Coronavirus 2019 from Colombia.

You can get the official dataset here: 
[INS - Official Report](https://www.datos.gov.co/Salud-y-Protecci-n-Social/Casos-positivos-de-COVID-19-en-Colombia/gt2j-8ykr)

The number of new cases are increasing day by day around the world.
This dataset has information about reported cases from 32 Colombia departments.

You can get the Google COVID-19 Community Mobility Reports - Colombia.

You can view and collaborate to the analysis here:
[colombia_covid_19_analysis](https://www.kaggle.com/sebaxtian/colombia-covid-19-analysis) Kaggle Notebook Kernel.

---

In [2]:
# Any results you write to the current directory are saved as output.
OUTPUT_DIR = './'
if os.path.split(os.path.abspath('.'))[-1] == 'src':
    OUTPUT_DIR = '../output'
# Official Opendata Report, with anomalies
#URL_DATASET = 'https://www.datos.gov.co/api/views/gt2j-8ykr/rows.csv?accessType=DOWNLOAD'

# Official Daily Report Until Now
URL_DATASET = 'https://infogram.com/api/live/flex/4524241a-91a7-4bbd-a58e-63c12fb2952f/7109db63-a83b-4ba7-b5bc-565b2a7ee4bb?'

---

## Load Local Covid19 Report from Output

In [3]:
# Load local covid19_df from output
covid19_df = pd.read_csv(os.path.join(OUTPUT_DIR, 'covid19.csv'))
# Report last day
covid19_df.shape

(3439, 14)

---

## Get Official Covid19 Daily Report

In [4]:
# Get Covid19 dataset daily report
# Reading the json as a dict
with requests.get(URL_DATASET) as original_dataset:
    data = original_dataset.json()
#print(data)

# Department Report
#print(data['data'][0])
#department_report = data['data'][0]

# National Report
#print(len(data['data'][1:]))
national_report = data['data'][1:]

# New Cases
new_cases = national_report[0]
#print(new_cases)

# Deaths
new_deaths = national_report[1]
#print(new_deaths)

# Recovered
new_recovered = national_report[2]
#print(new_recovered)

# Recovered Hostpital
new_recovered_hospital = national_report[3]
#print(new_recovered_hospital)


# Get attributes and data
attrs = new_cases[0]
del new_cases[0]
#print('new_cases', new_cases)

# Build dataframe
covid_df = pd.DataFrame(data=new_cases, columns=attrs)

# Size dataframe
covid_df.shape

(206, 9)

In [5]:
# Get Covid19 dataset
#covid_df = pd.read_csv(URL_DATASET)

In [6]:
# Show dataframe
covid_df.tail()

,Caso,Fecha de diagnóstico,Ciudad,Departamento,Ubicación,Edad,Sexo,Tipo,Pais de procedencia
201,3435,17/04/2020,Bogota,Bogotá D.C.,Hospital,60,M,En estudio,Colombia
202,3436,17/04/2020,Bogota,Bogotá D.C.,Hospital,61,M,En estudio,Colombia
203,3437,17/04/2020,Medellin,Antioquia,Casa,25,F,En estudio,Colombia
204,3438,17/04/2020,Quibdo,Chocó,Casa,24,F,En estudio,Colombia
205,3439,17/04/2020,Leticia,Amazonas,Casa,26,M,En estudio,Colombia


In [7]:
# Show original columns
#print(covid_df.columns.values)

In [8]:
# Rename columns
covid_df.rename(columns={
    "Caso": "id_case",
    "Fecha de diagnóstico": "date",
    "Ciudad": "city",
    "Departamento": "dept_dist",
    "Ubicación": "care",
    "Edad": "age",
    "Sexo": "sex",
    "Tipo": "kind",
    "Pais de procedencia": "country_origin"}, inplace=True)
# Show dataframe
covid_df.head()

,id_case,date,city,dept_dist,care,age,sex,kind,country_origin
0,3234,15/04/2020,Cartagena,Cartagena D.T. y C,Casa,43,F,En estudio,Colombia
1,3235,15/04/2020,Cali,Valle del Cauca,Casa,38,F,En estudio,Colombia
2,3236,15/04/2020,Cali,Valle del Cauca,Hospital,47,F,En estudio,Colombia
3,3237,18/03/2020,Pereira,Risaralda,Recuperado (Hospital),78,M,Importado,Estados Unidos
4,3238,15/04/2020,Marsella,Risaralda,Casa,35,F,Relacionado,Colombia


In [9]:
# Clean empty rows
covid_df = covid_df[covid_df['date'] != '']
# Show dataframe
covid_df.tail()

,id_case,date,city,dept_dist,care,age,sex,kind,country_origin
201,3435,17/04/2020,Bogota,Bogotá D.C.,Hospital,60,M,En estudio,Colombia
202,3436,17/04/2020,Bogota,Bogotá D.C.,Hospital,61,M,En estudio,Colombia
203,3437,17/04/2020,Medellin,Antioquia,Casa,25,F,En estudio,Colombia
204,3438,17/04/2020,Quibdo,Chocó,Casa,24,F,En estudio,Colombia
205,3439,17/04/2020,Leticia,Amazonas,Casa,26,M,En estudio,Colombia


In [10]:
# Remove accents
covid_df['city'] = covid_df['city'].transform(lambda value: unidecode.unidecode(value))
covid_df['dept_dist'] = covid_df['dept_dist'].transform(lambda value: unidecode.unidecode(value))
# Show dataframe
covid_df.head()

,id_case,date,city,dept_dist,care,age,sex,kind,country_origin
0,3234,15/04/2020,Cartagena,Cartagena D.T. y C,Casa,43,F,En estudio,Colombia
1,3235,15/04/2020,Cali,Valle del Cauca,Casa,38,F,En estudio,Colombia
2,3236,15/04/2020,Cali,Valle del Cauca,Hospital,47,F,En estudio,Colombia
3,3237,18/03/2020,Pereira,Risaralda,Recuperado (Hospital),78,M,Importado,Estados Unidos
4,3238,15/04/2020,Marsella,Risaralda,Casa,35,F,Relacionado,Colombia


In [11]:
# Setup date format
def setup_date(value):
    #print('Cosa', value)
    value = value.split('/')
    #print(value)
    # Check day
    if len(value[0]) == 1:
        value[0] = '0' + value[0]
    # Check mounth
    if len(value[1]) == 1:
        value[1] = '0' + value[1]
    # Check year
    if len(value[2]) == 2:
        value[2] = value[2] + '20'
    # Return new date format
    return '/'.join(value)
# Check date format
def check_date_format(value):
    try:
        new_date = datetime.datetime.strptime(value, '%d/%m/%Y')
        #print('Fecha Valida:', value)
        return new_date
    except ValueError:
        #print('Fecha con Error:', value)
        #new_date = datetime.datetime.now().strptime('%d/%m/%Y')
        return np.NaN

# Setup date format
covid_df['date'] = covid_df['date'].transform(lambda value: setup_date(value))
covid_df['date'] = covid_df['date'].transform(lambda value: check_date_format(value))
covid_df['date'].fillna(method='bfill', inplace=True)
# Setup date format
covid_df['date'] = [value.strftime('%d/%m/%Y') for value in pd.to_datetime(covid_df['date'], format='%d/%m/%Y')]
# Show dataframe
covid_df.tail()

,id_case,date,city,dept_dist,care,age,sex,kind,country_origin
201,3435,17/04/2020,Bogota,Bogota D.C.,Hospital,60,M,En estudio,Colombia
202,3436,17/04/2020,Bogota,Bogota D.C.,Hospital,61,M,En estudio,Colombia
203,3437,17/04/2020,Medellin,Antioquia,Casa,25,F,En estudio,Colombia
204,3438,17/04/2020,Quibdo,Choco,Casa,24,F,En estudio,Colombia
205,3439,17/04/2020,Leticia,Amazonas,Casa,26,M,En estudio,Colombia


In [12]:
# Add Day, Month, Year, Month Name and Day Name
covid_df['day'] = covid_df['date'].transform(lambda value: value.split('/')[0])
covid_df['month'] = covid_df['date'].transform(lambda value: value.split('/')[1])
covid_df['year'] = covid_df['date'].transform(lambda value: value.split('/')[2])
# English
#covid_df['month_name'] = covid_df['month'].transform(lambda value: month_name[int(value)])
#covid_df['day_name'] = covid_df['date'].transform(lambda value: day_name[date(int(value.split('/')[2]), int(value.split('/')[1]), int(value.split('/')[0])).weekday()])
# Spanish
nombre_mes = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
nombre_dia = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']
covid_df['month_name'] = covid_df['month'].transform(lambda value: nombre_mes[int(value) - 1])
covid_df['day_name'] = covid_df['date'].transform(lambda value: nombre_dia[date(int(value.split('/')[2]), int(value.split('/')[1]), int(value.split('/')[0])).weekday()])
# Show dataframe
covid_df.head()

,id_case,date,city,dept_dist,care,age,sex,kind,country_origin,day,month,year,month_name,day_name
0,3234,15/04/2020,Cartagena,Cartagena D.T. y C,Casa,43,F,En estudio,Colombia,15,04,2020,Abril,Miercoles
1,3235,15/04/2020,Cali,Valle del Cauca,Casa,38,F,En estudio,Colombia,15,04,2020,Abril,Miercoles
2,3236,15/04/2020,Cali,Valle del Cauca,Hospital,47,F,En estudio,Colombia,15,04,2020,Abril,Miercoles
3,3237,18/03/2020,Pereira,Risaralda,Recuperado (Hospital),78,M,Importado,Estados Unidos,18,03,2020,Marzo,Miercoles
4,3238,15/04/2020,Marsella,Risaralda,Casa,35,F,Relacionado,Colombia,15,04,2020,Abril,Miercoles


In [13]:
# Update Case ID
#covid_df['id_case'] = covid_df.index
#covid_df.tail()

In [14]:
# Sort columns
covid_df = covid_df[['id_case', 'date', 'day', 'month', 'year', 'month_name', 'day_name', 'city', 'dept_dist', 'age', 'sex', 'kind', 'country_origin', 'care']]
# Show dataframe
covid_df.head()

,id_case,date,day,month,year,month_name,day_name,city,dept_dist,age,sex,kind,country_origin,care
0,3234,15/04/2020,15,04,2020,Abril,Miercoles,Cartagena,Cartagena D.T. y C,43,F,En estudio,Colombia,Casa
1,3235,15/04/2020,15,04,2020,Abril,Miercoles,Cali,Valle del Cauca,38,F,En estudio,Colombia,Casa
2,3236,15/04/2020,15,04,2020,Abril,Miercoles,Cali,Valle del Cauca,47,F,En estudio,Colombia,Hospital
3,3237,18/03/2020,18,03,2020,Marzo,Miercoles,Pereira,Risaralda,78,M,Importado,Estados Unidos,Recuperado (Hospital)
4,3238,15/04/2020,15,04,2020,Abril,Miercoles,Marsella,Risaralda,35,F,Relacionado,Colombia,Casa


In [15]:
# Setup string title format
covid_df['city'] = covid_df['city'].transform(lambda value: value.title())
covid_df['dept_dist'] = covid_df['dept_dist'].transform(lambda value: value.title())
covid_df['country_origin'] = covid_df['country_origin'].transform(lambda value: value.title())
# Show dataframe
covid_df.tail()

,id_case,date,day,month,year,month_name,day_name,city,dept_dist,age,sex,kind,country_origin,care
201,3435,17/04/2020,17,04,2020,Abril,Viernes,Bogota,Bogota D.C.,60,M,En estudio,Colombia,Hospital
202,3436,17/04/2020,17,04,2020,Abril,Viernes,Bogota,Bogota D.C.,61,M,En estudio,Colombia,Hospital
203,3437,17/04/2020,17,04,2020,Abril,Viernes,Medellin,Antioquia,25,F,En estudio,Colombia,Casa
204,3438,17/04/2020,17,04,2020,Abril,Viernes,Quibdo,Choco,24,F,En estudio,Colombia,Casa
205,3439,17/04/2020,17,04,2020,Abril,Viernes,Leticia,Amazonas,26,M,En estudio,Colombia,Casa


## Merge Local Covid19 Report with Official Covid19 Daily Report

In [16]:
# Merge with the local covid19_df from output
# Temporal update id_case
#covid19_df['id_case'] = covid19_df['id_case'].transform(lambda value: value + 1) OJO
# Check changes
#print(covid_df['id_case'].values[0])
#print(covid19_df['id_case'].values[-1])
if int(covid_df['id_case'].values[0]) > int(covid19_df['id_case'].values[-1]):
    covid_df = pd.concat([covid19_df, covid_df])
else:
    covid_df = covid19_df
# Show dataframe
covid_df.shape
#covid_df.tail()

(3439, 14)

In [17]:
# Reset index
covid_df.reset_index(inplace=True)
covid_df.drop(columns=['index'], inplace=True)
# Show dataframe
covid_df.tail()

,id_case,date,day,month,year,month_name,day_name,city,dept_dist,age,sex,kind,country_origin,care
3434,3435,17/04/2020,17,4,2020,Abril,Viernes,Bogota,Bogota D.C.,60,M,En Estudio,Colombia,Hospital
3435,3436,17/04/2020,17,4,2020,Abril,Viernes,Bogota,Bogota D.C.,61,M,En Estudio,Colombia,Hospital
3436,3437,17/04/2020,17,4,2020,Abril,Viernes,Medellin,Antioquia,25,F,En Estudio,Colombia,Casa
3437,3438,17/04/2020,17,4,2020,Abril,Viernes,Quibdo,Choco,24,F,En Estudio,Colombia,Casa
3438,3439,17/04/2020,17,4,2020,Abril,Viernes,Leticia,Amazonas,26,M,En Estudio,Colombia,Casa


In [18]:
# Setup string title format
covid_df['kind'] = covid_df['kind'].transform(lambda value: value.title())
covid_df['care'] = covid_df['care'].transform(lambda value: value.title())
# Show dataframe
covid_df.tail()

,id_case,date,day,month,year,month_name,day_name,city,dept_dist,age,sex,kind,country_origin,care
3434,3435,17/04/2020,17,4,2020,Abril,Viernes,Bogota,Bogota D.C.,60,M,En Estudio,Colombia,Hospital
3435,3436,17/04/2020,17,4,2020,Abril,Viernes,Bogota,Bogota D.C.,61,M,En Estudio,Colombia,Hospital
3436,3437,17/04/2020,17,4,2020,Abril,Viernes,Medellin,Antioquia,25,F,En Estudio,Colombia,Casa
3437,3438,17/04/2020,17,4,2020,Abril,Viernes,Quibdo,Choco,24,F,En Estudio,Colombia,Casa
3438,3439,17/04/2020,17,4,2020,Abril,Viernes,Leticia,Amazonas,26,M,En Estudio,Colombia,Casa


## Covid 19 Dataset
> ***Output file***: covid19.csv

In [19]:
# Save dataframe
covid_df.to_csv(os.path.join(OUTPUT_DIR, 'covid19.csv'), index=False)

---

## Covid19 Daily Report Dataset Updated

In [20]:
# Show dataframe
covid_df.head()

,id_case,date,day,month,year,month_name,day_name,city,dept_dist,age,sex,kind,country_origin,care
0,1,06/03/2020,6,3,2020,Marzo,Viernes,Bogota,Bogota D.C.,19,F,Importado,Italia,Recuperado
1,2,09/03/2020,9,3,2020,Marzo,Lunes,Buga,Valle Del Cauca,34,M,Importado,España,Recuperado
2,3,09/03/2020,9,3,2020,Marzo,Lunes,Medellin,Antioquia,50,F,Importado,España,Recuperado
3,4,11/03/2020,11,3,2020,Marzo,Miercoles,Medellin,Antioquia,55,M,Relacionado,Colombia,Recuperado
4,5,11/03/2020,11,3,2020,Marzo,Miercoles,Medellin,Antioquia,25,M,Relacionado,Colombia,Recuperado


In [21]:
# Show dataframe
covid_df.tail()

,id_case,date,day,month,year,month_name,day_name,city,dept_dist,age,sex,kind,country_origin,care
3434,3435,17/04/2020,17,4,2020,Abril,Viernes,Bogota,Bogota D.C.,60,M,En Estudio,Colombia,Hospital
3435,3436,17/04/2020,17,4,2020,Abril,Viernes,Bogota,Bogota D.C.,61,M,En Estudio,Colombia,Hospital
3436,3437,17/04/2020,17,4,2020,Abril,Viernes,Medellin,Antioquia,25,F,En Estudio,Colombia,Casa
3437,3438,17/04/2020,17,4,2020,Abril,Viernes,Quibdo,Choco,24,F,En Estudio,Colombia,Casa
3438,3439,17/04/2020,17,4,2020,Abril,Viernes,Leticia,Amazonas,26,M,En Estudio,Colombia,Casa


---

## Samples Processed

In [22]:
# Samples Processed
# Reading the json as a dict
with requests.get('https://infogram.com/api/live/flex/4524241a-91a7-4bbd-a58e-63c12fb2952f/4dc066f9-6be5-4791-aefb-96fe4c523dfd?') as original_dataset:
    data = original_dataset.json()
#print(data['data'][0][0][0])

# Get attributes and data
attrs = data['data'][0][0][0]
del data
#print(attrs)
samples_processed = attrs.split('<b>')[1].split('</b>')[0].replace('.', '')
#print('Samples Processed:', samples_processed)

## Samples Descarted

In [23]:
# Descarted Cases
# Reading the json as a dict
with requests.get('https://infogram.com/api/live/flex/4524241a-91a7-4bbd-a58e-63c12fb2952f/4dc066f9-6be5-4791-aefb-96fe4c523dfd?') as original_dataset:
    data = original_dataset.json()
#print(data['data'][0][1][0])

# Get attributes and data
attrs = data['data'][0][1][0]
del data
#print(attrs)
samples_descarted = attrs.split('<b>')[1].split('</b>')[0].replace('.', '')
#print('Samples Descarted:', samples_descarted)

---

## Time Line Samples Processed

In [24]:
# Time Line Samples Processed
# Reading the json as a dict
with requests.get('https://infogram.com/api/live/flex/4524241a-91a7-4bbd-a58e-63c12fb2952f/0a859405-7279-44a1-80d9-ff127bdf4489?') as original_dataset:
    data = original_dataset.json()
#print(data['data'])
#print(data['data'][0])

# Get attributes and data
attrs = data['data'][0][0]
attrs[0] = 'Periodo'
del data['data'][0][0]
#print(attrs)
data = data['data'][0]
#print(data)

# Build dataframe
covid_df_samples_processed = pd.DataFrame(data=data, columns=attrs)

# Size dataframe
covid_df_samples_processed.tail()

,Periodo,Muestras procesadas,Acumulado procesadas
2,09/03/20-15/03/20,1584,2343
3,16/03/20-22/03/20,4818,7161
4,23/03/20-29/03/20,8903,16064
5,30/03/20-05/04/20,11621,27685
6,06/04/20-12/04/20,15302,42987


In [25]:
# Rename columns
covid_df_samples_processed.rename(columns={
    "Periodo": "period",
    "Muestras procesadas": "total_samples",
    "Acumulado procesadas": "accum_samples"}, inplace=True)
# Show dataframe
covid_df_samples_processed.tail()

,period,total_samples,accum_samples
2,09/03/20-15/03/20,1584,2343
3,16/03/20-22/03/20,4818,7161
4,23/03/20-29/03/20,8903,16064
5,30/03/20-05/04/20,11621,27685
6,06/04/20-12/04/20,15302,42987


In [26]:
# Update date format
def update_date_format(period):
    date1 = period.split('-')[0]
    date2 = period.split('-')[1]
    if date1.split('/')[-1] == '20':
        date1 = '/'.join(date1.split('/')[0:-1]) + '/2020'
    if date2.split('/')[-1] == '20':
        date2 = '/'.join(date2.split('/')[0:-1]) + '/2020'
    return date1 + '-' + date2
# Example
#update_date_format('02/03/20-08/03/20')
# Update date format
covid_df_samples_processed['period'] = covid_df_samples_processed['period'].transform(lambda value: update_date_format(value))
# Show dataframe
covid_df_samples_processed.tail()

,period,total_samples,accum_samples
2,09/03/2020-15/03/2020,1584,2343
3,16/03/2020-22/03/2020,4818,7161
4,23/03/2020-29/03/2020,8903,16064
5,30/03/2020-05/04/2020,11621,27685
6,06/04/2020-12/04/2020,15302,42987


## Time Line Samples Processed
> ***Output file***: covid19_samples_processed.csv

In [27]:
# Save dataframe
covid_df_samples_processed.to_csv(os.path.join(OUTPUT_DIR, 'covid19_samples_processed.csv'), index=False)

---

## Time Line by Care

In [28]:
# Care Classes
list(set(covid_df['care'].values))

['Recuperado (Hospital)',
 'Hospital Uci',
 'Recuperado',
 'Fallecido',
 'Casa',
 'Hospital']

In [29]:
# Get time line by care [cases, recovered, deaths]
def get_time_line_by_care(care):
    # covid_df_report
    covid_df_report = pd.DataFrame()
    # Check care
    if care == '*':
        covid_df_report = covid_df.groupby('date')['date'].count()
    elif care == 'Recuperado':
        covid_df_report = covid_df[(covid_df['care'] == 'Recuperado') | (covid_df['care'] == 'Recuperado (Hospital)')].groupby('date')['date'].count()
    else:
        covid_df_report = covid_df[covid_df['care'] == care].groupby('date')['date'].count()
    # ---
    # Create dateset
    covid_df_report = pd.DataFrame(data={'date': covid_df_report.index, 'total': covid_df_report.values}, columns=['date', 'total'])
    covid_df_report['date_iso'] = pd.to_datetime(covid_df_report['date'], format='%d/%m/%Y')
    covid_df_report = covid_df_report.sort_values(by=['date_iso'], ascending=True)
    covid_df_report['accum'] = covid_df_report['total'].cumsum()
    covid_df_report = covid_df_report.drop(columns=['date_iso'])
    covid_df_report.reset_index(inplace=True, drop=True)
    # Return
    return covid_df_report

# Time line by cases
covid_df_cases = get_time_line_by_care('*')
#print('covid_df_cases', '\n', covid_df_cases)

# Time line by recovered
covid_df_recovered = get_time_line_by_care('Recuperado')
#print('covid_df_recovered', '\n', covid_df_recovered.head())

# Time line by deaths
covid_df_deaths = get_time_line_by_care('Fallecido')
#print('covid_df_deaths', '\n', covid_df_deaths.head())

# Get total
def get_total(df_target, value):
    num = [i for i in range(len(df_target['date'].values)) if df_target['date'].values[i] == value]
    if num:
        return df_target['total'].values[num[0]]
    return 0

# Get accumulator
def get_accum(df_target, value):
    num = [i for i in range(len(df_target['date'].values)) if df_target['date'].values[i] == value]
    if num:
        return df_target['accum'].values[num[0]]
    return np.nan

# Time line by care [cases, recovered, deaths]
covid_19_time_line_by_care = pd.DataFrame(columns=['date', 'cases', 'accum_cases', 'recovered', 'accum_recovered', 'deaths', 'accum_deaths'])
covid_19_time_line_by_care['date'] = [dti.strftime('%d/%m/%Y') for dti in pd.date_range(start='2020-03-01', end=date.today().isoformat(), freq='D')]

# Cases
covid_19_time_line_by_care['cases'] = covid_19_time_line_by_care['date'].transform(lambda value: get_total(covid_df_cases, value))
covid_19_time_line_by_care['accum_cases'] = covid_19_time_line_by_care['date'].transform(lambda value: get_accum(covid_df_cases, value))
# Recovered
covid_19_time_line_by_care['recovered'] = covid_19_time_line_by_care['date'].transform(lambda value: get_total(covid_df_recovered, value))
covid_19_time_line_by_care['accum_recovered'] = covid_19_time_line_by_care['date'].transform(lambda value: get_accum(covid_df_recovered, value))
# Deaths
covid_19_time_line_by_care['deaths'] = covid_19_time_line_by_care['date'].transform(lambda value: get_total(covid_df_deaths, value))
covid_19_time_line_by_care['accum_deaths'] = covid_19_time_line_by_care['date'].transform(lambda value: get_accum(covid_df_deaths, value))

# Fill NaN values
covid_19_time_line_by_care.fillna(method='ffill', inplace=True)
covid_19_time_line_by_care.fillna(0, inplace=True)
#print(covid_19_time_line_by_care.dtypes)
covid_19_time_line_by_care = covid_19_time_line_by_care.astype({'accum_cases': 'int64', 'accum_recovered': 'int64', 'accum_deaths': 'int64'})

# Remove last day until update it
if covid_df.values[-1][1] != covid_19_time_line_by_care.values[-1][0]:
    covid_19_time_line_by_care.drop([covid_19_time_line_by_care.index[-1]], inplace=True)

# Show dataframe
covid_19_time_line_by_care.tail()

,date,cases,accum_cases,recovered,accum_recovered,deaths,accum_deaths
43,13/04/2020,76,2856,0,549,1,141
44,14/04/2020,129,2985,0,549,0,141
45,15/04/2020,149,3134,0,549,1,142
46,16/04/2020,200,3334,2,551,3,145
47,17/04/2020,105,3439,0,551,0,145


## Time Line by Care
> ***Output file***: covid19_time_line.csv

In [30]:
# Save dataframe
covid_19_time_line_by_care.to_csv(os.path.join(OUTPUT_DIR, 'covid19_time_line.csv'), index=False)

---

In [31]:
# Care Classes
list(set(covid_df['care'].values))

['Recuperado (Hospital)',
 'Hospital Uci',
 'Recuperado',
 'Fallecido',
 'Casa',
 'Hospital']

## Time Line by Care by City

In [32]:
# Time Line by Care by City
cities = []
cities = list(set(covid_df['city'].values))

# Get report by care and city [cases, recovered, deaths]
def get_report_by_care_city(care, city):
    # covid_df_report
    covid_df_report = pd.DataFrame()
    # Check care
    if care == '*':
        covid_df_report = covid_df[covid_df['city'] == city].groupby('date')['date'].count()
    elif care == 'Recuperado':
        covid_df_report = covid_df[((covid_df['care'] == 'Recuperado') | (covid_df['care'] == 'Recuperado (Hospital)')) & (covid_df['city'] == city)].groupby('date')['date'].count()
    else:
        covid_df_report = covid_df[(covid_df['care'] == care) & (covid_df['city'] == city)].groupby('date')['date'].count()
    # Create dateset
    covid_df_report = pd.DataFrame(data={'date': covid_df_report.index, 'total': covid_df_report.values}, columns=['date', 'total'])
    covid_df_report['date_iso'] = pd.to_datetime(covid_df_report['date'], format='%d/%m/%Y')
    covid_df_report = covid_df_report.sort_values(by=['date_iso'], ascending=True)
    covid_df_report['accum'] = covid_df_report['total'].cumsum()
    covid_df_report = covid_df_report.drop(columns=['date_iso'])
    covid_df_report.reset_index(inplace=True, drop=True)
    # Return
    return covid_df_report

# Get time line by care and city
def get_time_line_by_care_city(city):
    # Time line by cases and city
    covid_df_cases = get_report_by_care_city('*', city)
    # Time line by recovered and city
    covid_df_recovered = get_report_by_care_city('Recuperado', city)
    # Time line by deaths and city
    covid_df_deaths = get_report_by_care_city('Fallecido', city)

    # Time line by care and city [cases, recovered, deaths]
    covid_19_time_line_by_care_city = pd.DataFrame(columns=['date', 'cases', 'accum_cases', 'recovered', 'accum_recovered', 'deaths', 'accum_deaths'])
    covid_19_time_line_by_care_city['date'] = [dti.strftime('%d/%m/%Y') for dti in pd.date_range(start='2020-03-01', end=date.today().isoformat(), freq='D')]

    # Cases
    covid_19_time_line_by_care_city['cases'] = covid_19_time_line_by_care_city['date'].transform(lambda value: get_total(covid_df_cases, value))
    covid_19_time_line_by_care_city['accum_cases'] = covid_19_time_line_by_care_city['date'].transform(lambda value: get_accum(covid_df_cases, value))

    # Recovered
    covid_19_time_line_by_care_city['recovered'] = covid_19_time_line_by_care_city['date'].transform(lambda value: get_total(covid_df_recovered, value))
    covid_19_time_line_by_care_city['accum_recovered'] = covid_19_time_line_by_care_city['date'].transform(lambda value: get_accum(covid_df_recovered, value))

    # Deaths
    covid_19_time_line_by_care_city['deaths'] = covid_19_time_line_by_care_city['date'].transform(lambda value: get_total(covid_df_deaths, value))
    covid_19_time_line_by_care_city['accum_deaths'] = covid_19_time_line_by_care_city['date'].transform(lambda value: get_accum(covid_df_deaths, value))

    # Fill NaN values
    covid_19_time_line_by_care_city.fillna(method='ffill', inplace=True)
    covid_19_time_line_by_care_city.fillna(0, inplace=True)
    #print(covid_19_time_line_by_care_city.dtypes)
    covid_19_time_line_by_care_city = covid_19_time_line_by_care_city.astype({'accum_cases': 'int64', 'accum_recovered': 'int64', 'accum_deaths': 'int64'})

    # Remove last day until update it
    if covid_df.values[-1][1] != covid_19_time_line_by_care_city.values[-1][0]:
        covid_19_time_line_by_care_city.drop([covid_19_time_line_by_care_city.index[-1]], inplace=True)

    # Return dataframe
    return covid_19_time_line_by_care_city


# By city
covid_19_time_line_by_care_city = {}
for city in cities:
    #print('city', city)
    # Get time line by care and city
    covid_19_time_line_by_care_city[city] = get_time_line_by_care_city(city)


## Time Line by Care and City
> ***Output file***: covid19_time_line_city.csv

In [33]:
# Save dataframe
for city in covid_19_time_line_by_care_city:
    #print('city:', city)
    # Save dataframe
    covid_19_time_line_by_care_city[city].to_csv(os.path.join(OUTPUT_DIR, 'covid19_time_line_city_' + city + '.csv'), index=False)

---

In [34]:
# Care Classes
list(set(covid_df['care'].values))

['Recuperado (Hospital)',
 'Hospital Uci',
 'Recuperado',
 'Fallecido',
 'Casa',
 'Hospital']

## Time Line by Care by Department or District

In [35]:
# Time Line by Care by Department or District
depts_dists = []
depts_dists = list(set(covid_df['dept_dist'].values))

# Get report by care and dept_dist [cases, recovered, deaths]
def get_report_by_care_dept_dist(care, dept_dist):
    # covid_df_report
    covid_df_report = pd.DataFrame()
    # Check care
    if care == '*':
        covid_df_report = covid_df[covid_df['dept_dist'] == dept_dist].groupby('date')['date'].count()
    elif care == 'Recuperado':
        covid_df_report = covid_df[((covid_df['care'] == 'Recuperado') | (covid_df['care'] == 'Recuperado (Hospital)')) & (covid_df['dept_dist'] == dept_dist)].groupby('date')['date'].count()
    else:
        covid_df_report = covid_df[(covid_df['care'] == care) & (covid_df['dept_dist'] == dept_dist)].groupby('date')['date'].count()
    # Create dateset
    covid_df_report = pd.DataFrame(data={'date': covid_df_report.index, 'total': covid_df_report.values}, columns=['date', 'total'])
    covid_df_report['date_iso'] = pd.to_datetime(covid_df_report['date'], format='%d/%m/%Y')
    covid_df_report = covid_df_report.sort_values(by=['date_iso'], ascending=True)
    covid_df_report['accum'] = covid_df_report['total'].cumsum()
    covid_df_report = covid_df_report.drop(columns=['date_iso'])
    covid_df_report.reset_index(inplace=True, drop=True)
    # Return
    return covid_df_report

# Get time line by care and dept_dist
def get_time_line_by_care_dept_dist(dept_dist):
    # Time line by cases and dept_dist
    covid_df_cases = get_report_by_care_dept_dist('*', dept_dist)
    # Time line by recovered and dept_dist
    covid_df_recovered = get_report_by_care_dept_dist('Recuperado', dept_dist)
    # Time line by deaths and dept_dist
    covid_df_deaths = get_report_by_care_dept_dist('Fallecido', dept_dist)

    # Time line by care and dept_dist [cases, recovered, deaths]
    covid_19_time_line_by_care_dept_dist = pd.DataFrame(columns=['date', 'cases', 'accum_cases', 'recovered', 'accum_recovered', 'deaths', 'accum_deaths'])
    covid_19_time_line_by_care_dept_dist['date'] = [dti.strftime('%d/%m/%Y') for dti in pd.date_range(start='2020-03-01', end=date.today().isoformat(), freq='D')]

    # Cases
    covid_19_time_line_by_care_dept_dist['cases'] = covid_19_time_line_by_care_dept_dist['date'].transform(lambda value: get_total(covid_df_cases, value))
    covid_19_time_line_by_care_dept_dist['accum_cases'] = covid_19_time_line_by_care_dept_dist['date'].transform(lambda value: get_accum(covid_df_cases, value))

    # Recovered
    covid_19_time_line_by_care_dept_dist['recovered'] = covid_19_time_line_by_care_dept_dist['date'].transform(lambda value: get_total(covid_df_recovered, value))
    covid_19_time_line_by_care_dept_dist['accum_recovered'] = covid_19_time_line_by_care_dept_dist['date'].transform(lambda value: get_accum(covid_df_recovered, value))

    # Deaths
    covid_19_time_line_by_care_dept_dist['deaths'] = covid_19_time_line_by_care_dept_dist['date'].transform(lambda value: get_total(covid_df_deaths, value))
    covid_19_time_line_by_care_dept_dist['accum_deaths'] = covid_19_time_line_by_care_dept_dist['date'].transform(lambda value: get_accum(covid_df_deaths, value))

    # Fill NaN values
    covid_19_time_line_by_care_dept_dist.fillna(method='ffill', inplace=True)
    covid_19_time_line_by_care_dept_dist.fillna(0, inplace=True)
    #print(covid_19_time_line_by_care_dept_dist.dtypes)
    covid_19_time_line_by_care_dept_dist = covid_19_time_line_by_care_dept_dist.astype({'accum_cases': 'int64', 'accum_recovered': 'int64', 'accum_deaths': 'int64'})

    # Remove last day until update it
    if covid_df.values[-1][1] != covid_19_time_line_by_care_dept_dist.values[-1][0]:
        covid_19_time_line_by_care_dept_dist.drop([covid_19_time_line_by_care_dept_dist.index[-1]], inplace=True)

    # Return dataframe
    return covid_19_time_line_by_care_dept_dist


# By dept_dist
covid_19_time_line_by_care_dept_dist = {}
for dept_dist in depts_dists:
    #print('dept_dist', dept_dist)
    # Get time line by care and dept_dist
    covid_19_time_line_by_care_dept_dist[dept_dist] = get_time_line_by_care_dept_dist(dept_dist)


## Time Line by Care and Department or District
> ***Output file***: covid19_time_line_dept_dist.csv

In [36]:
# Save dataframe
for dept_dist in covid_19_time_line_by_care_dept_dist:
    #print('dept_dist:', dept_dist)
    # Save dataframe
    covid_19_time_line_by_care_dept_dist[dept_dist].to_csv(os.path.join(OUTPUT_DIR, 'covid19_time_line_dept_dist_' + dept_dist + '.csv'), index=False)

---

## Google Community Mobility Reports - Colombia

In [37]:
# Google Community Mobility Reports - Colombia
google_community_mobility_reports = pd.DataFrame(columns=['date', 'country', 'file', 'url'])
google_community_mobility_reports['date'] = [dti.strftime('%Y-%m-%d') for dti in pd.date_range(start='2020-03-29', end=date.today().isoformat(), freq='D')]
google_community_mobility_reports['country'] = ['Colombia' for country in range(len(google_community_mobility_reports['date'].values))]
google_community_mobility_reports['file'] = [ date + '_CO_Mobility_Report_en.pdf' for date in google_community_mobility_reports['date'].values]
# Get URL report
def get_report_url(file):
    with requests.get('https://www.gstatic.com/covid19/mobility/' + file) as community_mobility_report:
        #print(community_mobility_report.status_code)
        if community_mobility_report.status_code == 200:
            #print('status_code: ', 200)
            #print('url', community_mobility_report.url)
            return community_mobility_report.url
        else:
            return np.nan
# Get URL report
google_community_mobility_reports['url'] = google_community_mobility_reports['file'].transform(lambda value: get_report_url(value))
# Drop any report without URL
google_community_mobility_reports.dropna(inplace=True)
# Reset index
google_community_mobility_reports.reset_index(inplace=True, drop=True)
# Show dataframe
google_community_mobility_reports.head()
#print('community_mobility_report.content', community_mobility_report.content)
            #with open(os.path.join(OUTPUT_DIR, '2020-04-05_CO_Mobility_Report_en.pdf'), 'wb') as f:
            #    f.write(community_mobility_report.content)

,date,country,file,url
0,2020-03-29,Colombia,2020-03-29_CO_Mobility_Report_en.pdf,https://www.gstatic.com/covid19/mobility/2020-...
1,2020-04-05,Colombia,2020-04-05_CO_Mobility_Report_en.pdf,https://www.gstatic.com/covid19/mobility/2020-...
2,2020-04-11,Colombia,2020-04-11_CO_Mobility_Report_en.pdf,https://www.gstatic.com/covid19/mobility/2020-...


In [38]:
# Add Mobility Changes
#from pdfminer.high_level import extract_pages
# Get mobility changes
def get_mobility_changes(URL):
    # Target changes
    targets = ['Retail & recreation', 'Grocery & pharmacy', 'Parks', 'Transit stations', 'Workplaces', 'Residential']
    # Mobility Changes
    mobility_changes = []
    # Get Mobility Report
    with requests.get(URL) as mobility_report:
        #print(mobility_report.status_code)
        if mobility_report.status_code == 200:
            temp = tempfile.NamedTemporaryFile()
            temp.write(mobility_report.content)
            #print('temp.name:', temp.name)
            with open(temp.name, 'rb') as file:
                # By pages
                pdf_text = []
                page = 0
                while page != -1:
                    text = extract_text(file, maxpages=1, page_numbers=[page])
                    if text:
                        #print('text', text)
                        pdf_text.append(text.split('\n'))
                        page += 1
                    else:
                        page = -1
                # Pages
                #print('Pages:', len(pdf_text))
                # Page 1
                page1 = pdf_text[0]
                page1 = filter(lambda value: value != '', page1)
                page1 = filter(lambda value: value in targets or value[-1] == '%', list(page1))
                page1 = list(page1)[:6]
                # Page 2
                page2 = pdf_text[1]
                page2 = filter(lambda value: value != '', page2)
                page2 = filter(lambda value: value in targets or value[-1] == '%', list(page2))
                page2 = list(page2)[:6]
                # Merge
                mobility_changes = page1 + page2
    return mobility_changes
# Add Mobility Changes
google_community_mobility_reports['mobility_changes'] = google_community_mobility_reports['url'].transform(lambda value: get_mobility_changes(value))
# By case
google_community_mobility_reports['Retail & recreation'] = google_community_mobility_reports['mobility_changes'].transform(lambda value: value[1])
google_community_mobility_reports['Grocery & pharmacy'] = google_community_mobility_reports['mobility_changes'].transform(lambda value: value[3])
google_community_mobility_reports['Parks'] = google_community_mobility_reports['mobility_changes'].transform(lambda value: value[5])
google_community_mobility_reports['Transit stations'] = google_community_mobility_reports['mobility_changes'].transform(lambda value: value[7])
google_community_mobility_reports['Workplaces'] = google_community_mobility_reports['mobility_changes'].transform(lambda value: value[9])
google_community_mobility_reports['Residential'] = google_community_mobility_reports['mobility_changes'].transform(lambda value: value[11])
# Drop column
google_community_mobility_reports.drop(columns=['mobility_changes'], inplace=True)
# Sort columns
google_community_mobility_reports = google_community_mobility_reports[['date', 'country', 'Retail & recreation', 'Grocery & pharmacy', 'Parks', 'Transit stations', 'Workplaces', 'Residential', 'file', 'url']]
# Setup date format
google_community_mobility_reports['date'] = [value.strftime('%d/%m/%Y') for value in pd.to_datetime(google_community_mobility_reports['date'], format='%Y-%m-%d')]
# Show dataframe
google_community_mobility_reports.head()

,date,country,Retail & recreation,Grocery & pharmacy,Parks,Transit stations,Workplaces,Residential,file,url
0,29/03/2020,Colombia,-86%,-68%,-80%,-82%,-58%,+26%,2020-03-29_CO_Mobility_Report_en.pdf,https://www.gstatic.com/covid19/mobility/2020-...
1,05/04/2020,Colombia,-85%,-66%,-79%,-81%,-57%,+26%,2020-04-05_CO_Mobility_Report_en.pdf,https://www.gstatic.com/covid19/mobility/2020-...
2,11/04/2020,Colombia,-84%,-65%,-77%,-81%,-69%,+32%,2020-04-11_CO_Mobility_Report_en.pdf,https://www.gstatic.com/covid19/mobility/2020-...


## Google COVID-19 Community Mobility Reports - Colombia
> ***Output file***: google_community_mobility_reports.csv

In [39]:
# Save dataframe
google_community_mobility_reports.to_csv(os.path.join(OUTPUT_DIR, 'google_community_mobility_reports.csv'), index=False)

---

In [40]:
# Insights Colombia
# Reading the json as a dict
with requests.get('https://infogram.com/api/live/flex/4524241a-91a7-4bbd-a58e-63c12fb2952f/90df9eae-e3e5-4982-b71c-3e28e3c4273f?') as original_dataset:
    data = original_dataset.json()
#print(data['data'][0])
#print(data['data'][0][1][0])

total_cases = data['data'][0][0][0]
#print(total_cases)
total_cases = total_cases.split('<b>')[1].split('</b>')[0].replace('.', '')

total_recovered = data['data'][0][1][0]
#print(total_recovered)
total_recovered = total_recovered.split('<b>')[1].split('</b>')[0].replace('.', '')

total_deaths = data['data'][0][2][0]
#print(total_deaths)
total_deaths = total_deaths.split('<b>')[1].split('</b>')[0].replace('.', '')

#print('Total Cases:', total_cases)
#print('Total Recovered:', total_recovered)
#print('Total Deaths:', total_deaths)

In [41]:
# Type of Case
# Reading the json as a dict
with requests.get('https://infogram.com/api/live/flex/4524241a-91a7-4bbd-a58e-63c12fb2952f/a991990b-3b23-4ab2-8452-0ed1162d4896?') as original_dataset:
    data = original_dataset.json()
#print(data['data'][0])
#print(data['data'][0][0])

type_imported = data['data'][0][0][0]

type_related = data['data'][0][1][0]

type_study = data['data'][0][2][0]


#print('Total Importado:', type_imported)
#print('Total Relacionado:', type_related)
#print('Total Estudio:', type_study)

## Resume

In [42]:
# Resume
data = []
# cases_by_care_by_date[N] = ['Hospital', 'Hospital UCI', 'Casa', 'Fallecido', 'Recuperado', 'Recuperado (Hospital)']
# Resume Attributes
data.append(['Confirmados', total_cases])
data.append(['Recuperados', total_recovered])
data.append(['Fallecidos', total_deaths])
data.append(['Muestras descartadas', samples_descarted])
data.append(['Importado', type_imported])
data.append(['Relacionado', type_related])
data.append(['En estudio', type_study])
data.append(['Muestras procesadas', samples_processed])

# Resume Dataframe
covid_df_resume = pd.DataFrame(data=data, columns=['title', 'total'])
# Show dataframe
covid_df_resume.head(10)

,title,total
0,Confirmados,3439
1,Recuperados,634
2,Fallecidos,153
3,Casos descartados,52134
4,Importado,779
5,Relacionado,1160
6,En estudio,1500
7,Muestras procesadas,55573


## Resume
> ***Output file***: covid19_resume.csv

In [43]:
# Save dataframe
covid_df_resume.to_csv(os.path.join(OUTPUT_DIR, 'covid19_resume.csv'), index=False)
print('\nColombia Covid19 Resumen:')
print(covid_df_resume)


Colombia Covid19 Resumen:
                 title  total
0          Confirmados   3439
1          Recuperados    634
2           Fallecidos    153
3    Casos descartados  52134
4            Importado    779
5          Relacionado   1160
6           En estudio   1500
7  Muestras procesadas  55573


---